# 参数初始化
参数初始化对模型具有较大的影响，不同的初始化方式可能会导致截然不同的结果，所幸的是很多深度学习的先驱们已经帮我们探索了各种各样的初始化方式，所以我们只需要学会如何对模型的参数进行初始化的赋值即可。

PyTorch 的初始化方式并没有那么显然，如果你使用最原始的方式创建模型，那么你需要定义模型中的所有参数，当然这样你可以非常方便地定义每个变量的初始化方式，但是对于复杂的模型，这并不容易，而且我们推崇使用 Sequential 和 Module 来定义模型，所以这个时候我们就需要知道如何来自定义初始化方式

## 使用 NumPy 来初始化
因为 PyTorch 是一个非常灵活的框架，理论上能够对所有的 Tensor 进行操作，所以我们能够通过定义新的 Tensor 来初始化，直接看下面的例子

In [1]:
import numpy as np
import torch
from torch import nn

In [2]:
# 定义一个 Sequential 模型
net1 = nn.Sequential(
    nn.Linear(30, 40),
    nn.ReLU(),
    nn.Linear(40, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [3]:
# 访问第一层的参数
w1 = net1[0].weight
b1 = net1[0].bias

In [4]:
print(w1)

Parameter containing:
tensor([[ 0.0423,  0.0642,  0.1742,  ..., -0.1704,  0.1794, -0.0647],
        [ 0.1689, -0.0612,  0.1166,  ..., -0.1203, -0.1132,  0.0982],
        [ 0.0142, -0.1803, -0.0662,  ..., -0.1253,  0.1711,  0.1172],
        ...,
        [-0.0204,  0.0469,  0.1407,  ..., -0.0891,  0.1818,  0.1602],
        [-0.0762, -0.1687,  0.1440,  ..., -0.1204,  0.1446, -0.1138],
        [ 0.0322, -0.1162,  0.0912,  ..., -0.1348, -0.1241,  0.0561]])


注意，这是一个 Parameter，也就是一个特殊的 Variable，我们可以访问其 `.data`属性得到其中的数据，然后直接定义一个新的 Tensor 对其进行替换，我们可以使用 PyTorch 中的一些随机数据生成的方式，比如 `torch.randn`，如果要使用更多 PyTorch 中没有的随机化方式，可以使用 numpy

In [5]:
# 定义一个 Tensor 直接对其进行替换
net1[0].weight.data = torch.from_numpy(np.random.uniform(3, 5, size=(40, 30)))

In [6]:
print(net1[0].weight)

Parameter containing:
tensor([[ 3.2409,  4.2360,  3.4090,  ...,  4.6374,  4.0964,  3.8584],
        [ 3.7140,  3.1968,  4.8042,  ...,  4.6313,  4.8988,  4.0364],
        [ 4.9183,  4.4929,  3.6181,  ...,  4.1351,  3.0997,  3.7327],
        ...,
        [ 3.4762,  3.2713,  4.4487,  ...,  3.5780,  3.0948,  3.4981],
        [ 4.3567,  4.7754,  3.9867,  ...,  3.4380,  3.7446,  3.0836],
        [ 3.2425,  4.4053,  3.7900,  ...,  4.3053,  4.5621,  4.7163]], dtype=torch.float64)


可以看到这个参数的值已经被改变了，也就是说已经被定义成了我们需要的初始化方式，如果模型中某一层需要我们手动去修改，那么我们可以直接用这种方式去访问，但是更多的时候是模型中相同类型的层都需要初始化成相同的方式，这个时候一种更高效的方式是使用循环去访问，比如

In [7]:
for layer in net1:
    if isinstance(layer, nn.Linear): # 判断是否是线性层
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 
        # 定义为均值为 0，方差为 0.5 的正态分布

**小练习：一种非常流行的初始化方式叫 Xavier，方法来源于 2010 年的一篇论文 [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a.html)，其通过数学的推到，证明了这种初始化方式可以使得每一层的输出方差是尽可能相等的，有兴趣的同学可以去看看论文**

我们给出这种初始化的公式

$$
w\ \sim \ Uniform[- \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}, \frac{\sqrt{6}}{\sqrt{n_j + n_{j+1}}}]
$$

其中 $n_j$ 和 $n_{j+1}$ 表示该层的输入和输出数目，所以请尝试实现以下这种初始化方式

对于 Module 的参数初始化，其实也非常简单，如果想对其中的某层进行初始化，可以直接像 Sequential 一样对其 Tensor 进行重新定义，其唯一不同的地方在于，如果要用循环的方式访问，需要介绍两个属性，children 和 modules，下面我们举例来说明

In [8]:
class sim_net(nn.Module):
    def __init__(self):
        super(sim_net, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(30, 40),
            nn.ReLU()
        )
        
        self.l1[0].weight.data = torch.randn(40, 30) # 直接对某一层初始化
        
        self.l2 = nn.Sequential(
            nn.Linear(40, 50),
            nn.ReLU()
        )
        
        self.l3 = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.l1(x)
        x =self.l2(x)
        x = self.l3(x)
        return x

In [9]:
net2 = sim_net()

In [10]:
# 访问 children
for i in net2.children():
    print(i)

Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)


In [11]:
# 访问 modules
for i in net2.modules():
    print(i)

sim_net(
  (l1): Sequential(
    (0): Linear(in_features=30, out_features=40, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=40, out_features=50, bias=True)
    (1): ReLU()
  )
  (l3): Sequential(
    (0): Linear(in_features=50, out_features=10, bias=True)
    (1): ReLU()
  )
)
Sequential(
  (0): Linear(in_features=30, out_features=40, bias=True)
  (1): ReLU()
)
Linear(in_features=30, out_features=40, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=40, out_features=50, bias=True)
  (1): ReLU()
)
Linear(in_features=40, out_features=50, bias=True)
ReLU()
Sequential(
  (0): Linear(in_features=50, out_features=10, bias=True)
  (1): ReLU()
)
Linear(in_features=50, out_features=10, bias=True)
ReLU()


通过上面的例子，看到区别了吗?

children 只会访问到模型定义中的第一层，因为上面的模型中定义了三个 Sequential，所以只会访问到三个 Sequential，而 modules 会访问到最后的结构，比如上面的例子，modules 不仅访问到了 Sequential，也访问到了 Sequential 里面，这就对我们做初始化非常方便，比如

In [12]:
for layer in net2.modules():
    if isinstance(layer, nn.Linear):
        param_shape = layer.weight.shape
        layer.weight.data = torch.from_numpy(np.random.normal(0, 0.5, size=param_shape)) 

这上面实现了和 Sequential 相同的初始化，同样非常简便

## torch.nn.init
因为 PyTorch 灵活的特性，我们可以直接对 Tensor 进行操作从而初始化，PyTorch 也提供了初始化的函数帮助我们快速初始化，就是 `torch.nn.init`，其操作层面仍然在 Tensor 上，下面我们举例说明

In [13]:
from torch.nn import init

In [14]:
print(net1[0].weight)

Parameter containing:
tensor([[ 0.6217,  0.1405, -0.6086,  ...,  0.6905,  0.5743, -0.0957],
        [-0.7869,  1.1874, -0.1948,  ...,  0.3092, -1.5781, -0.6207],
        [-0.2711, -0.5583, -0.0548,  ...,  0.1185,  0.6405, -0.5375],
        ...,
        [-0.2878, -0.6291,  0.2910,  ..., -0.3549, -0.0533, -0.4994],
        [ 0.2760,  0.3957,  0.4414,  ...,  0.4492, -0.5281, -0.7638],
        [-1.0145,  0.7051, -0.5610,  ...,  0.2178, -0.7287,  0.6156]], dtype=torch.float64)


In [17]:
init.xavier_uniform_(net1[0].weight) # 这就是上面我们讲过的 Xavier 初始化方法，PyTorch 直接内置了其实现

Parameter containing:
tensor([[ 0.2643,  0.0615,  0.0417,  ...,  0.2701,  0.1728, -0.1023],
        [ 0.1062, -0.1466,  0.2825,  ..., -0.2471, -0.1994, -0.2611],
        [-0.2867, -0.2175, -0.2920,  ...,  0.1644, -0.0541, -0.2734],
        ...,
        [ 0.1424,  0.0746,  0.2334,  ..., -0.1088,  0.0071, -0.1954],
        [ 0.0389, -0.0236,  0.2310,  ..., -0.0118,  0.0094,  0.1354],
        [ 0.0223, -0.1418,  0.2443,  ..., -0.1326, -0.0386, -0.2663]], dtype=torch.float64)

In [18]:
print(net1[0].weight)

Parameter containing:
tensor([[ 0.2643,  0.0615,  0.0417,  ...,  0.2701,  0.1728, -0.1023],
        [ 0.1062, -0.1466,  0.2825,  ..., -0.2471, -0.1994, -0.2611],
        [-0.2867, -0.2175, -0.2920,  ...,  0.1644, -0.0541, -0.2734],
        ...,
        [ 0.1424,  0.0746,  0.2334,  ..., -0.1088,  0.0071, -0.1954],
        [ 0.0389, -0.0236,  0.2310,  ..., -0.0118,  0.0094,  0.1354],
        [ 0.0223, -0.1418,  0.2443,  ..., -0.1326, -0.0386, -0.2663]], dtype=torch.float64)


可以看到参数已经被修改了

`torch.nn.init` 为我们提供了更多的内置初始化方式，避免了我们重复去实现一些相同的操作

上面讲了两种初始化方式，其实它们的本质都是一样的，就是去修改某一层参数的实际值，而 `torch.nn.init` 提供了更多成熟的深度学习相关的初始化方式，非常方便

下一节课，我们将讲一下目前流行的各种基于梯度的优化算法